# Explore the National Water Model Reanalysis v2.1 


### CASE 1: each HDF file contains chunks in multiple variables; doesn't need h5py

370k files.

In [ ]:
%%time
import fsspec
import xarray as xr
mapper = fsspec.get_mapper(
    "reference://",
    fo="s3://esip-qhub-public/noaa/nwm/nwm_reanalysis.json.zst",
    target_options={
        "anon": True,
        "compression": "zstd"
    },
    remote_options={"anon": True}
)
ds = xr.open_dataset(mapper, engine="zarr", backend_kwargs={"consolidated": False})

In [ ]:
ds

In [ ]:
ds.nbytes/1e12, ds.streamflow.nbytes/1e12 # How many terabytes total/in a single variable

#### Read and plot streamflow for a specific time 
The local National Weather Service office in Houston observed all-time record daily rainfall accumulations on both August 26 and 27, measured at 14.4 in (370 mm) and 16.08 in (408 mm) respectively

In [ ]:
import hvplot.pandas
import geoviews as gv
from holoviews.operation.datashader import rasterize
import cartopy.crs as ccrs
import numpy as np
import pandas as pd

In [ ]:
ds1 = ds.sel(time='2017-08-27 18:00:00', method='nearest')

In [ ]:
var = 'streamflow'

In [ ]:
df = ds1[var].to_pandas().to_frame()

In [ ]:
date_title = pd.to_datetime(ds1.time.values).strftime('%Y-%m-%d %H-%M-%D')
date_title = f'{var}: {date_title}'
date_title

In [ ]:
df = df.assign(latitude=ds.latitude)
df = df.assign(longitude=ds.longitude)
df.rename(columns={0: var}, inplace=True)

In [ ]:
p = df.hvplot.points(x='longitude', y='latitude', geo=True,
                     c=var, colorbar=True, size=14, label=date_title)
g = rasterize(p, aggregator='mean', x_sampling=0.02, 
                y_sampling=0.02, width=500).opts(tools=['hover'], 
                aspect='equal', logz=True, cmap='viridis', clim=(1e-2, np.nan))
g * gv.tile_sources.OSM